# Google Ads Campaign Performance Optimization

```SQL
DROP TABLE IF EXISTS dim_campaign;

DROP TABLE IF EXISTS fct_ad_performance;

CREATE TABLE dim_campaign (
    campaign_id INTEGER,
    segment VARCHAR
);

CREATE TABLE fct_ad_performance (
    campaign_id INTEGER,
    ad_format VARCHAR,
    impressions INTEGER,
    clicks INTEGER,
    cost FLOAT,
    revenue FLOAT,
    campaign_date DATE
);

INSERT INTO dim_campaign (campaign_id, segment)
VALUES
    (101, 'Tech Ads'),
    (102, 'Retail Growth'),
    (103, 'Finance Solutions'),
    (104, 'Health & Wellness'),
    (105, 'Automotive'),
    (106, 'Education'),
    (107, 'Travel'),
    (108, 'Entertainment'),
    (109, 'Food & Beverage'),
    (110, 'Real Estate');

INSERT INTO fct_ad_performance (
    campaign_id,
    ad_format,
    impressions,
    clicks,
    cost,
    revenue,
    campaign_date
)
VALUES
    (101, 'video', 500, 50, 100.0, 150.0, '2024-07-10'),
    (101, 'display', 300, 30, 60.0, 90.0, '2024-07-20'),
    (102, 'search', 400, 40, 80.0, 120.0, '2024-07-15'),
    (103, 'video', 600, 55, 120.0, 200.0, '2024-07-18'),
    (104, 'shopping', 200, 20, 40.0, 70.0, '2024-07-22'),
    (105, 'search', 350, 35, 70.0, 110.0, '2024-07-05'),
    (106, 'display', 450, 45, 90.0, 130.0, '2024-07-11'),
    (101, 'video', 1200, 80, 200.0, 300.0, '2024-08-03'),
    (102, 'search', 800, 60, 150.0, 180.0, '2024-08-04'),
    (103, 'display', 1100, 70, 180.0, 250.0, '2024-08-09'),
    (104, 'video', 950, 65, 170.0, 210.0, '2024-08-12'),
    (105, 'shopping', 1300, 85, 220.0, 330.0, '2024-08-15'),
    (106, 'search', 1050, 75, 190.0, 260.0, '2024-08-22'),
    (107, 'video', 500, 45, 100.0, 140.0, '2024-08-27'),
    (108, 'display', 1150, 78, 210.0, 310.0, '2024-08-05'),
    (109, 'search', 700, 50, 120.0, 200.0, '2024-09-10'),
    (110, 'display', 600, 45, 90.0, 150.0, '2024-09-15'),
    (101, 'shopping', 800, 60, 160.0, 240.0, '2024-09-20'),
    (102, 'video', 900, 80, 180.0, 280.0, '2024-09-21'),
    (103, 'search', 650, 55, 130.0, 190.0, '2024-09-22');

SELECT * FROM dim_campaign;
SELECT * FROM fct_ad_performance;
```

In [3]:
import pandas as pd
import numpy as np

In [4]:
df_campaign = pd.read_csv('Data/005/dim_campaign_2.csv')
df_performance = pd.read_csv('Data/005/fct_ad_performance.csv', parse_dates=['campaign_date'])

df_campaign.head()

,campaign_id,segment
0,101,Tech Ads
1,102,Retail Growth
2,103,Finance Solutions
3,104,Health & Wellness
4,105,Automotive


In [4]:
df_performance.head()

,campaign_id,ad_format,impressions,clicks,cost,revenue,campaign_date
0,101,video,500,50,100,150,2024-07-10
1,101,display,300,30,60,90,2024-07-20
2,102,search,400,40,80,120,2024-07-15
3,103,video,600,55,120,200,2024-07-18
4,104,shopping,200,20,40,70,2024-07-22


# Pregunta 1

### Para cada segmento de campaña publicitaria, ¿cuáles son los formatos de anuncios únicos utilizados durante julio de 2024? Esto nos ayudará a comprender la diversidad en nuestros formatos de anuncios

In [2]:
df_merge = df_campaign.merge(df_performance, on='campaign_id', how='inner')

df_julio = df_merge[
    (df_merge['campaign_date'].between('2024-07-01','2024-07-31'))
]

diversidad_formatos = df_julio[['segment','ad_format']].drop_duplicates().sort_values(by='segment')

diversidad_formatos

NameError: name 'df_campaign' is not defined

```SQL
SELECT
    DISTINCT c.segment,
    f.ad_format
FROM fct_ad_performance f
INNER JOIN dim_campaign c ON c.campaign_id = f.campaign_id
WHERE f.campaign_date BETWEEN '2024-07-01' AND '2024-07-31'
ORDER BY  c.segment;
```

# Pregunta 2

¿Cuántas campañas únicas tuvieron al menos un periodo móvil de 7 días en agosto de 2024 donde sus impresiones totales superaron las 1,000? Queremos identificar las campañas que tuvieron un alto alcance en al menos una ventana de 7 días durante este mes.

In [6]:
df_merge = df_campaign.merge(df_performance, on='campaign_id', how='inner')
# 1. Filtramos y ordenamos por fecha (vital para sumas móviles)
df_aug = df_merge[df_merge['campaign_date'].between('2024-08-01', '2024-08-31')].copy()
df_aug = df_aug.sort_values(['campaign_id', 'campaign_date'])

# 2. Calculamos la suma móvil de 7 días por cada campaña
df_aug['rolling_impressions'] = df_aug.groupby('campaign_id')['impressions'].transform(
    lambda x: x.rolling(window=7, min_periods=1).sum()
)

# 3. Contamos cuántas campañas únicas superaron las 1,000 impresiones
campañas_exitosas = df_aug[df_aug['rolling_impressions'] > 1000]['campaign_id'].nunique()

campañas_exitosas

5

```SQL
SELECT COUNT(DISTINCT campaign_id)
FROM(
    SELECT
        campaign_id,
        SUM(impressions) OVER(
            PARTITION BY campaign_id
            ORDER BY campaign_date
            ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
            ) as rolling_sum
    FROM fct_ad_performance
    WHERE campaign_date BETWEEN '2024-08-01' AND '2024-08-31') as temporal
WHERE rolling_sum > 1000;
```

# Pregunta 3

"¿Cuál es el ROI total para cada segmento de campaña en el tercer trimestre (Q3) de 2024? Y, ¿cómo se compara con el ROI promedio de todas las campañas (devuelve las etiquetas 'higher than average' o 'lower than average')? Usaremos esto para identificar qué segmentos están superando el promedio.

- Nota 1: El ROI se define como $(Revenue - Cost) / Cost$.
- Nota 2: Para el ROI promedio entre segmentos, calcula primero el ROI por segmento y luego calcula el promedio de esos resultados."

In [7]:
# 1. Filtro Q3 y Agregación por Segmento
df_q3 = df_merge[df_merge['campaign_date'].between('2024-07-01', '2024-09-30')]
resumen = df_q3.groupby('segment').agg({'revenue': 'sum', 'cost': 'sum'})

# 2. Calcular ROI por Segmento
resumen['roi'] = (resumen['revenue'] - resumen['cost']) / resumen['cost']

# 3. Calcular el promedio de los ROIs
avg_roi = resumen['roi'].mean()

# 4. Crear la etiqueta comparativa (Benchmarking)
resumen['performance_label'] = resumen['roi'].apply(
    lambda x: 'higher than average' if x > avg_roi else 'lower than average'
)

resumen

,revenue,cost,roi,performance_label
segment,,,,
Automotive,440,290,0.517241,higher than average
Education,390,280,0.392857,lower than average
Entertainment,310,210,0.476190,lower than average
Finance Solutions,640,430,0.488372,higher than average
Food & Beverage,200,120,0.666667,higher than average
Health & Wellness,280,210,0.333333,lower than average
Real Estate,150,90,0.666667,higher than average
Retail Growth,580,410,0.414634,lower than average
Tech Ads,780,520,0.500000,higher than average


```SQL
WITH Segment_ROI AS (
    -- Paso 1: ROI por segmento
    SELECT 
        c.segment,
        SUM(f.revenue) as total_rev,
        SUM(f.cost) as total_cost,
        (SUM(f.revenue) - SUM(f.cost)) / NULLIF(SUM(f.cost), 0) as roi
    FROM fct_ad_performance f
    JOIN dim_campaign c ON f.campaign_id = c.campaign_id
    WHERE f.campaign_date BETWEEN '2024-07-01' AND '2024-09-30'
    GROUP BY c.segment
),
Global_Avg AS (
    -- Paso 2: Promedio de los ROIs de los segmentos
    SELECT AVG(roi) as avg_roi FROM Segment_ROI
)
-- Paso 3: Comparación final
SELECT 
    s.segment,
    s.roi,
    CASE 
        WHEN s.roi > g.avg_roi THEN 'higher than average'
        ELSE 'lower than average'
    END as performance_label
FROM Segment_ROI s, Global_Avg g;
```